# Pre-processing of BGT data from PDOK

In [ ]:
# Helper script to allow importing from parent folder.
import set_path

import src.utils.las_utils as las_utils
import src.utils.pdok_utils as pdok_utils

In [ ]:
# Specify the bounding box of the area we want to scrape.
# For example, we can find the bounding box of a folder of LAS files.
bbox_padded = las_utils.get_bbox_from_las_folder('../datasets/pointcloud/', padding=10)

# Specify the tight bounding box. For some reason, the PDOK scraper returns a bigger area than the one requested, 
# so we use this one later to precisely select the desired objects.
bbox = las_utils.get_bbox_from_las_folder('../datasets/pointcloud/', padding=0)

# Output folder to store the files.
bgt_data_folder = '../datasets/bgt/'

In [ ]:
# Scrape data from PDOK, this will return a download link.
download_url = pdok_utils.scrape_pdok_bgt(bbox=bbox_padded, bgt_layers=["paal", "pand", "vegetatieobject"])

In [ ]:
# Extract the downloaded archive.
local_filename = download_url.split('/')[-1]

!mkdir -p {bgt_data_folder}
!wget {download_url} -P {bgt_data_folder}
!unzip -o {bgt_data_folder + local_filename} -d {bgt_data_folder}
!rm {bgt_data_folder + local_filename}

In [ ]:
# The vegetation_file, pole_file, and building_file are those we just downloaded.
vegetation_file = bgt_data_folder + 'bgt_vegetatieobject.gml'
pole_file = bgt_data_folder + 'bgt_paal.gml'
building_file = bgt_data_folder + 'bgt_pand.gml'

# Parse the downloaded CityGML files.
tree_point_file = pdok_utils.parse_vegetation(vegetation_file, bbox=bbox, out_folder=bgt_data_folder)
pole_point_file = pdok_utils.parse_poles(pole_file, bbox=bbox, out_folder=bgt_data_folder)
building_shape_file = pdok_utils.parse_buildings(building_file, bbox=bbox, out_folder=bgt_data_folder)

# Merge the point objects to a single csv file.
point_file = pdok_utils.merge_point_files((tree_point_file, pole_point_file), out_folder=bgt_data_folder)

# Clean-up
!rm {vegetation_file} {pole_file} {building_file} {tree_point_file} {pole_point_file}
del [vegetation_file, pole_file, building_file, tree_point_file, pole_point_file]

In [ ]:
# Load the data for visualization.
import csv
import ast

buildings = []

with open(bgt_data_folder + 'bgt_buildings.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        buildings.append(ast.literal_eval(row['Polygon']))
        
trees = []
poles = []

with open(bgt_data_folder + 'bgt_points.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Type'] == 'boom':
            trees.append((float(row['X']), float(row['Y'])))
        else:
            poles.append((float(row['X']), float(row['Y'])))

In [ ]:
# Plot the data.
%matplotlib widget
from matplotlib import pyplot as plt
import matplotlib.patches as patches

from shapely.geometry import Polygon
polygons = [Polygon(poly) for poly in buildings]

fig, ax = plt.subplots(1)

for polygon in polygons:
    x, y = polygon.exterior.xy
    plt.fill(x, y, edgecolor='red', facecolor='lightgrey')

for tree in trees:
    plt.scatter(tree[0], tree[1], c='green')

for pole in poles:
    plt.scatter(pole[0], pole[1], c='purple', marker='x')

((x_min, y_max), (x_max, y_min)) = bbox
box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='k', fill=False)
ax.add_patch(box)

plt.axis('equal')
plt.show()